In [1]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hnmchallenge.dataset import Dataset
from hnmchallenge.filtered_dataset import FilterdDataset
from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.constant import *
from hnmchallenge.models.sgmc.sgmc import SGMC
from hnmchallenge.models.ease.ease import EASE
from hnmchallenge.models.itemknn.itemknn import ItemKNN 

In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [3]:
dataset = FilterdDataset()
dr = DataReader()

In [4]:
users_mb = dataset.get_filtered_user_multiply_buy()
normal_users = users_mb[users_mb["diff"] == 0][DEFAULT_USER_COL]
users_top_pop = users_mb[~(users_mb["diff"] == 0)][DEFAULT_USER_COL]

In [33]:
normal_users

2                2
4                4
5                5
6                6
9                9
            ...   
1136199    1136199
1136200    1136200
1136202    1136202
1136203    1136203
1136205    1136205
Name: customer_id, Length: 631395, dtype: int64

In [34]:
users_top_pop

0                0
1                1
3                3
7                7
8                8
            ...   
1136177    1136177
1136181    1136181
1136197    1136197
1136201    1136201
1136204    1136204
Name: customer_id, Length: 504811, dtype: int64

In [ ]:
recom = SGMC(dataset, k=128, time_weight=True)

In [5]:
recom = ItemKNN(dataset, topk=1000, time_weight=True)

In [ ]:
recom = EASE(dataset, l2=1e-3, time_weight=False)

In [24]:
train = dataset.get_train_df()
train_sub = dataset.get_train_df_user_subset()
fd = dr.get_filtered_full_data()

In [7]:
recom.compute_similarity_matrix(train)  

INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 1128716


func:interactions_to_sparse_matrix
 took: 8.67193055152893 sec


In [25]:
normal_users_train = train[train[DEFAULT_USER_COL].isin(normal_users)]
mb_users_train = train[train[DEFAULT_USER_COL].isin(users_top_pop)]

In [26]:
len(train[DEFAULT_USER_COL].unique())

1128716

In [27]:
len(normal_users_train[DEFAULT_USER_COL].unique())

625091

In [28]:
len(mb_users_train[DEFAULT_USER_COL].unique())

503625

In [35]:
val_df_users = dataset.get_val_df()[DEFAULT_USER_COL].unique()
test_df_users = dataset.get_test_df()[DEFAULT_USER_COL].unique()

normal_users_val_train = train[(train[DEFAULT_USER_COL].isin(val_df_users)) & (train[DEFAULT_USER_COL].isin(normal_users))]
normal_users_test_train = train[(train[DEFAULT_USER_COL].isin(test_df_users)) & (train[DEFAULT_USER_COL].isin(normal_users))]
normal_users_train = pd.concat([normal_users_val_train, normal_users_test_train], axis=0)

mb_users_val_train = train[(train[DEFAULT_USER_COL].isin(val_df_users)) & (train[DEFAULT_USER_COL].isin(users_top_pop))]
mb_users_test_train = train[(train[DEFAULT_USER_COL].isin(test_df_users)) & (train[DEFAULT_USER_COL].isin(users_top_pop))]
mb_users_train = pd.concat([mb_users_val_train, mb_users_test_train], axis=0)

In [48]:
len(val_df_users)

37741

In [49]:
len(test_df_users)

37740

In [37]:
len(train[(train[DEFAULT_USER_COL].isin(val_df_users)) & (train[DEFAULT_USER_COL].isin(normal_users))][DEFAULT_USER_COL].unique())

8975

In [39]:
len(train[(train[DEFAULT_USER_COL].isin(test_df_users)) & (train[DEFAULT_USER_COL].isin(normal_users))][DEFAULT_USER_COL].unique())

8849

In [40]:
len(train[(train[DEFAULT_USER_COL].isin(val_df_users)) & (train[DEFAULT_USER_COL].isin(users_top_pop))][DEFAULT_USER_COL].unique())

25058

In [50]:
len(train[(train[DEFAULT_USER_COL].isin(test_df_users)) & (train[DEFAULT_USER_COL].isin(users_top_pop))][DEFAULT_USER_COL].unique())

25109

In [51]:
len(normal_users)

631395

In [52]:
len(users_top_pop)

504811

In [45]:
25058*2

50116

In [36]:
len(normal_users_train[DEFAULT_USER_COL].unique())

17824

In [33]:
len(mb_users_train[DEFAULT_USER_COL].unique())

50167

In [30]:
normal_users_recs = recom.recommend_multicore(interactions=normal_users_train, batch_size=10_000, num_cpus=20,
                                 remove_seen=False)

INFO:hnmchallenge.recommender_interface:Recommending items MULTICORE


  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
map_at_k(rating_true=dataset.get_val_df(), rating_pred=normal_users_recs, adjust_user_count=False)

0.003294871248631694

In [11]:
map_at_k(rating_true=dataset.get_test_df(), rating_pred=normal_users_recs, adjust_user_count=False)

0.003164769951695028

In [12]:
mb_users_recs = recom.recommend_multicore(interactions=mb_users_train, batch_size=10_000, num_cpus=20,
                                 remove_seen=False)

INFO:hnmchallenge.recommender_interface:Recommending items MULTICORE


  0%|          | 0/6 [00:00<?, ?it/s]

Predicting using time_weight importance...


INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 8362
INFO:hnmchallenge.utils.sparse_matrix:Applying time weight on user-item interactions


func:interactions_to_sparse_matrix
 took: 0.4129347801208496 sec


...

INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
Process ForkPoolWorker-41:
Process ForkPoolWorker-44:
Process ForkPoolWorker-43:
Process ForkPoolWorker-42:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/pytho

Predicting using time_weight importance...


INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 8912
INFO:hnmchallenge.utils.sparse_matrix:Applying time weight on user-item interactions


func:interactions_to_sparse_matrix
 took: 0.1592862606048584 sec


INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True


In [26]:
map_at_k(rating_true=dataset.get_val_df(), rating_pred=mb_users_recs, adjust_user_count=False)

0.04340751067410986

In [27]:
map_at_k(rating_true=dataset.get_test_df(), rating_pred=mb_users_recs, adjust_user_count=False)

0.042822899681914536

In [9]:
recom.compute_similarity_matrix(fd)  

INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 1136206


func:interactions_to_sparse_matrix
 took: 9.251797199249268 sec


In [7]:
normal_users_fd = fd[fd[DEFAULT_USER_COL].isin(normal_users)]
mb_users_fd = fd[fd[DEFAULT_USER_COL].isin(users_top_pop)]

In [10]:
normal_users_recs_fd = recom.recommend_multicore(interactions=normal_users_fd, batch_size=10_000, num_cpus=20,
                                 remove_seen=True)

INFO:hnmchallenge.recommender_interface:Recommending items MULTICORE


  0%|          | 0/64 [00:00<?, ?it/s]

Predicting using time_weight importance...


INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 9866
INFO:hnmchallenge.utils.sparse_matrix:Applying time weight on user-item interactions


func:interactions_to_sparse_matrix
 took: 0.15819191932678223 sec


...

INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...
INFO:hnmchallenge.recommender_interface:Removing seen items
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True


Predicting using time_weight importance...


INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 9865
INFO:hnmchallenge.utils.sparse_matrix:Applying time weight on user-item interactions


func:interactions_to_sparse_matrix
 took: 0.11440038681030273 sec


INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...
INFO:hnmchallenge.recommender_interface:Removing seen items
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
INFO:hnmchallenge.recommender_interface:Removing seen items
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True


In [11]:
mb_users_recs_fd = recom.recommend_multicore(interactions=mb_users_fd, batch_size=10_000, num_cpus=20,
                                 remove_seen=False)

INFO:hnmchallenge.recommender_interface:Recommending items MULTICORE


  0%|          | 0/51 [00:00<?, ?it/s]

Predicting using time_weight importance...


INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 9899
INFO:hnmchallenge.utils.sparse_matrix:Applying time weight on user-item interactions


func:interactions_to_sparse_matrix
 took: 0.3677823543548584 sec


...

INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...


Predicting using time_weight importance...


INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 9898
INFO:hnmchallenge.utils.sparse_matrix:Applying time weight on user-item interactions


func:interactions_to_sparse_matrix
 took: 0.14727163314819336 sec


INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True


In [12]:
from hnmchallenge.submission_handler import SubmissionHandler
sh = SubmissionHandler()                                               

In [13]:
sh.create_submission_filtered_data([normal_users_recs_fd, mb_users_recs_fd], sub_name="clusters_itemknn")

INFO:hnmchallenge.submission_handler:Submission with Filtered Data: clusters_itemknn created succesfully!
